In [ ]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Load data
df = pd.read_csv("credit_card_fraud_dataset.csv")  # or your actual file

# Convert date and extract features
df["TransactionDate"] = pd.to_datetime(df["TransactionDate"])
df["Hour"] = df["TransactionDate"].dt.hour
df["Day"] = df["TransactionDate"].dt.day
df["Month"] = df["TransactionDate"].dt.month
df["Weekday"] = df["TransactionDate"].dt.weekday
df.drop("TransactionDate", axis=1, inplace=True)

# Label Encoding for categorical features
label_encoders = {}
for col in ["TransactionType", "Location"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Split features and label (IMPORTANT: Exclude IsFraud from features)
X = df.drop("IsFraud", axis=1)
y = df["IsFraud"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Train model
model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))
print("\n📉 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# Save model and encoders
joblib.dump(model, "fraud_model.pkl")
joblib.dump(label_encoders, "label_encoders.pkl")
